# Bottleneck Simulated Annealing

In [ ]:
import numpy as np
import scipy.spatial
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

## Generate some data

In [ ]:
NUM_POINTS = 100
NUM_FEATURES = 12

### Generate points

In [ ]:
points = np.random.rand(NUM_POINTS, NUM_FEATURES)

### Compute pairwise distances

In [ ]:
NORM_P = 0.5
distance_matrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(points, "minkowski", p=NORM_P))
sns.displot(distance_matrix.flatten())

### Experiment: Determining a good distance metric for various dimensionalities

In [ ]:
import itertools
import pandas as pd

num_points = 200
num_features_space = [1, 2, 10, 100, 1000]
points_map = {num_features: np.random.rand(num_points, num_features) for num_features in num_features_space}

distance_metrics = ['braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'euclidean', 'jensenshannon', 'matching', 'minkowski', 'seuclidean', 'sqeuclidean']
distances = pd.DataFrame([
    {
        "num_features": num_features,
        "metric": distance_metric,
        "distance": distance
    }
    for (num_features, points), distance_metric in itertools.product(points_map.items(), distance_metrics)
    for distance in scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(points, distance_metric)).flatten()
])
grid = sns.FacetGrid((distances), hue="num_features", col="metric", col_wrap=4, sharex=False, sharey=False)
grid.map(sns.kdeplot, "distance", warn_singular=False)
grid.add_legend()

## Compute the best path through the points

### Define the objective

In [ ]:
from simanneal import Annealer

rng = np.random.default_rng()

class BottleneckAnnealer(Annealer):
    copy_strategy = "method"  # Use `self.state.copy()` to copy the state
    
    # Override default hyperparameters
    Tmin = 1e-6
    Tmax = 1e3
    steps = 250000
    updates = 100

    def __init__(self, *args, path: np.ndarray, distance_matrix: np.ndarray, **kwargs):
        super().__init__(path, *args, **kwargs)
        self.distance_matrix = distance_matrix
        self.num_points = len(self.state)

    def move(self):
        """
        Randomly swap points
        """
        num_swaps = int(np.random.rand() * 4)
        swaps = rng.choice(self.num_points, (num_swaps, 2), replace=False)
        self.state[swaps] = self.state[swaps[..., ::-1]]

    def energy(self):
        """
        Compute the energy of the current path
        """
        # Find the length of the edge from each node in the path to the next
        source_nodes = self.state
        target_nodes = np.roll(source_nodes, 1)
        edge_distances = self.distance_matrix[source_nodes, target_nodes]
        max_edge_length = edge_distances.max()
        mean_edge_length = edge_distances.mean()
        energy = max_edge_length + mean_edge_length
        return energy

### Initialize the annealer

In [ ]:
annealer = BottleneckAnnealer(path=np.arange(len(points)), distance_matrix=distance_matrix)
print(f"Start energy: {annealer.energy()}")

### Optimize

In [ ]:
best_path, best_energy = annealer.anneal()
print(f"Best energy: {best_energy}")
print(best_path)